# Tel Aviv Metropolitian LRT Network Isochrones
## How good is our coverage going to be, anyway?

Created by [Elad Alfassa](https://eladalfassa.com)

Based on [isochrone example](https://github.com/gboeing/osmnx-examples/blob/main/notebooks/13-isolines-isochrones.ipynb) by [Geoff Boeing](https://geoffboeing.com/)

In [1]:
# Basic setup
import json
from tqdm.notebook import tqdm
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
from descartes import PolygonPatch
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import Polygon
import shapely.geometry

%matplotlib widget
ox.config(log_console=True)
ox.__version__

'1.1.1'

In [2]:
# configure basic parameters
 # (Download the entire metropolitian area, and then some. Too bad OSM doesn't have a relationship for the Tel Aviv Metropoliain Area)
place = ["Tel Aviv District, Israel", "Center District, Israel"]
network_type = "walk"
trip_times = [5, 10, 15]  # in minutes
travel_speed = 4.5  # very approximate walking speed in km/hour (real humans might walk slower or faster)

## Download and prep the street network

In [3]:
# download the street network
graph = ox.graph_from_place(place, network_type=network_type)

In [4]:
# add an edge attribute for time in minutes required to traverse each edge
meters_per_minute = travel_speed * 1000 / 60  # km per hour to m per minute
for _, _, _, data in graph.edges(data=True, keys=True):
    data["time"] = data["length"] / meters_per_minute

## Load the station locations geojson
Originally downloaded from [geo.mot.gov.il](geo.mot.gov.il), converted into geojson using QGIS

In [5]:
lrt_stations_geojson = {}
with open('./data/tlv_lrt_stations.geojson') as f:
    lrt_stations_geojson = json.load(f)

# Collect nodes cloesest to each LRT station
lrt_station_nodes = []
for feature in tqdm(lrt_stations_geojson['features'], 'loading stations'):
    coords = feature['geometry']['coordinates']
    properties = feature['properties']
    node = ox.distance.nearest_nodes(graph, float(coords[0]), float(coords[1]))
    lrt_station_nodes.append({ "node": node, "properties": properties })
print(f'loaded {len(lrt_station_nodes)} LRT station locations')

metro_stations_geojson = {}
with open('./data/tlv_metro_stations.geojson') as f:
    metro_stations_geojson = json.load(f)

# Collect nodes cloesest to each metro station
metro_station_nodes = []
for feature in tqdm(metro_stations_geojson['features'], 'loading stations'):
    coords = feature['geometry']['coordinates']
    properties = feature['properties']
    node = ox.distance.nearest_nodes(graph, float(coords[0]), float(coords[1]))
    metro_station_nodes.append({ "node": node, "properties": properties })
print(f'loaded {len(metro_station_nodes)} metro station locations')

loading stations:   0%|          | 0/190 [00:00<?, ?it/s]

loaded 190 LRT station locations


loading stations:   0%|          | 0/110 [00:00<?, ?it/s]

loaded 110 metro station locations


## Generate the isochrones

In [6]:
# project the graph to UTM
# gdf_nodes = ox.graph_to_gdfs(graph, edges=False)
projected_graph = ox.project_graph(graph)

# get one color for each isochrone
iso_colors = ox.plot.get_colors(n=len(trip_times), cmap="plasma", start=0, return_hex=True)

In [7]:
# This function makes the isochrones, will be reused later
def make_iso_polys(G, center_node, edge_buff=25, node_buff=50, infill=False):
    isochrone_polys = {}
    for trip_time in sorted(trip_times, reverse=True):
        subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance="time")

        node_points = [Point((data["x"], data["y"])) for node, data in subgraph.nodes(data=True)]
        nodes_gdf = gpd.GeoDataFrame({"id": list(subgraph.nodes)}, geometry=node_points)
        nodes_gdf = nodes_gdf.set_index("id")

        edge_lines = []
        for n_fr, n_to in subgraph.edges():
            f = nodes_gdf.loc[n_fr].geometry
            t = nodes_gdf.loc[n_to].geometry
            edge_lookup = G.get_edge_data(n_fr, n_to)[0].get("geometry", LineString([f, t]))
            edge_lines.append(edge_lookup)

        n = nodes_gdf.buffer(node_buff).geometry
        e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
        all_gs = list(n) + list(e)
        new_iso = gpd.GeoSeries(all_gs).unary_union

        # try to fill in surrounded areas so shapes will appear solid and
        # blocks without white space inside them
        if infill and hasattr(new_iso, 'exterior'):
            new_iso = Polygon(new_iso.exterior)
        isochrone_polys[trip_time] = new_iso
    return isochrone_polys

def get_geojson_geometry(polygon):
    """Get geojson-compatible geometry, projected to a useful CRS"""
    return shapely.geometry.mapping(ox.projection.project_geometry(polygon, crs=projected_graph.graph['crs'], to_latlong=True)[0])


In [8]:
# Generate isochrone polygons, grouped by time
lrt_time_polys = { key: [] for key in trip_times }

for station in tqdm(lrt_station_nodes, 'making polys'):
    isochrone_polys = make_iso_polys(projected_graph, station['node'], edge_buff=25, node_buff=0, infill=True)
    for time, polygon in isochrone_polys.items():
        lrt_time_polys[time].append({ 'poly': polygon, 'properties': { **station['properties'], 'time': time }})


making polys:   0%|          | 0/190 [00:00<?, ?it/s]

## Draw and save the isochrones

In [9]:
z_index = { key: value for key, value in zip(trip_times, sorted(trip_times, reverse=True)) }

fig, ax = ox.plot_graph(
    projected_graph, show=False, close=False, edge_color="#999999", edge_alpha=0.2, node_size=0
)

geojson_features = []

for [time, polys], fc in zip(reversed(lrt_time_polys.items()), iso_colors):
    print(f'drawing for {time}')
    for polygon in tqdm(polys, 'drawing'):
        if polygon['poly']:
            geojson_features.append({"type": "Feature", "properties": { **polygon['properties'], "time": time }, "geometry": get_geojson_geometry(polygon['poly'])})
            patch = PolygonPatch(polygon['poly'], fc=fc, ec="none", alpha=0.7, zorder=z_index[time])
            ax.add_patch(patch)

# Save geojson
geojson = { "type": "FeatureCollection", "features": geojson_features }
with open('./tlv_lrt_isochrones_unmerged.geojson', 'w') as f:
    json.dump(geojson, f)

# show the plot
plt.show()

### Draw and save merged isochrones

In [10]:
fig, ax = ox.plot_graph(
    projected_graph, show=False, close=False, edge_color="#999999", edge_alpha=0.2, node_size=0
)

geojson_features = []

for [time, polys], fc in zip(reversed(lrt_time_polys.items()), iso_colors):
    print(f'drawing for {time}')
    union = shapely.ops.unary_union([poly['poly'] for poly in polys])
    for polygon in union:
        geojson_features.append({"type": "Feature", "properties": { "time": time, "color": fc }, "geometry": get_geojson_geometry(polygon)})
        patch = PolygonPatch(polygon, fc=fc, ec="none", alpha=0.7, zorder=z_index[time])
        ax.add_patch(patch)

geojson = { "type": "FeatureCollection", "features": geojson_features }
with open('./tlv_lrt_isochrones_merged.geojson', 'w') as f:
    json.dump(geojson, f)

plt.show()

## Generate metro station isochrones

(same code as LRT! just different source)

In [11]:
metro_time_polys = { key: [] for key in trip_times }

for station in tqdm(metro_station_nodes, 'making polys'):
    isochrone_polys = make_iso_polys(projected_graph, station['node'], edge_buff=25, node_buff=0, infill=True)
    for time, polygon in isochrone_polys.items():
        metro_time_polys[time].append({ 'poly': polygon, 'properties': { **station['properties'], 'time': time }})


making polys:   0%|          | 0/110 [00:00<?, ?it/s]

In [12]:
fig, ax = ox.plot_graph(
    projected_graph, show=False, close=False, edge_color="#999999", edge_alpha=0.2, node_size=0
)

geojson_features = []

for [time, polys], fc in zip(reversed(metro_time_polys.items()), iso_colors):
    print(f'drawing for {time}')
    for polygon in polys:
        if polygon['poly']:
            geojson_features.append({"type": "Feature", "properties": { **polygon['properties'], "time": time }, "geometry": get_geojson_geometry(polygon['poly'])})
            patch = PolygonPatch(polygon['poly'], fc=fc, ec="none", alpha=0.7, zorder=z_index[time])
            ax.add_patch(patch)
        else:
            print(polygon)

# Save geojson
geojson = { "type": "FeatureCollection", "features": geojson_features }
with open('./tlv_metro_isochrones_unmerged.geojson', 'w') as f:
    json.dump(geojson, f)

# show the plot
plt.show()

In [13]:
fig, ax = ox.plot_graph(
    projected_graph, show=False, close=False, edge_color="#999999", edge_alpha=0.2, node_size=0
)

geojson_features = []

for [time, polys], fc in zip(reversed(metro_time_polys.items()), iso_colors):
    print(f'drawing for {time}')
    union = shapely.ops.unary_union([poly['poly'] for poly in polys])
    for polygon in union:
        geojson_features.append({"type": "Feature", "properties": { "time": time, "color": fc }, "geometry": get_geojson_geometry(polygon)})
        patch = PolygonPatch(polygon, fc=fc, ec="none", alpha=0.7, zorder=z_index[time])
        ax.add_patch(patch)

geojson = { "type": "FeatureCollection", "features": geojson_features }
with open('./tlv_metro_isochrones_merged.geojson', 'w') as f:
    json.dump(geojson, f)

plt.show()

## For the web app only

Generate merged polygons in geojson for all possible group combos.

While I could use filtering on the client side, it won't work well because polygons of different lines / stations overlap and make the display more confusing, and I don't want to do merging in client side. 

In [24]:
# Define which files we're going to generate
lrt = ['אדום', 'סגול', 'ירוק']
metro =  ['M1', 'M2', 'M3']
brown_line = ['חום'] # the lonely BRT

combos = {
    'lrt': lrt,
    'metro': metro,
    'metro_lrt': metro + lrt,
    'metro_lrt_brown': metro + lrt + brown_line,
    'lrt_brown': lrt + brown_line,
    'brown_metro': metro + brown_line,
    'brown': brown_line,
}

In [15]:
# Combine the time_polys structure
combined_time_polys = { time: [] for time in trip_times }
for time in trip_times:
    combined_time_polys[time] = metro_time_polys[time] + lrt_time_polys[time]


In [16]:
for combo, combo_filter in combos.items():
    print(f'now running for {combo}')
    geojson_features = []

    def filter_poly(polygon):
        line = polygon['properties']['LINE'].split('-')[0]
        return line in combo_filter
    
    for [time, polys], fc in zip(reversed(combined_time_polys.items()), iso_colors):
        print(f'generating {time} minutes geojson features')
        filtered = filter(filter_poly, polys)
        union = shapely.ops.unary_union([poly['poly'] for poly in filtered])
        for polygon in tqdm(union):
            geojson_features.append({"type": "Feature", "properties": { "time": time, "color": fc }, "geometry": get_geojson_geometry(polygon)})

    geojson = { "type": "FeatureCollection", "features": geojson_features }
    with open(f'./tlv_{combo}_isochrones_merged.geojson', 'w') as f:
        json.dump(geojson, f)


now running for brown_metro
generating 15 minutes geojson features


  0%|          | 0/18 [00:00<?, ?it/s]

generating 10 minutes geojson features


  0%|          | 0/43 [00:00<?, ?it/s]

generating 5 minutes geojson features


  0%|          | 0/113 [00:00<?, ?it/s]

In [17]:
combos

{'brown_metro': ['M1', 'M2', 'M3', 'חום']}

In [25]:
# start by merging the base groups polygons, to speed up merging later
unioncache = { time: {} for time in trip_times }
for base_group in ['lrt', 'metro', 'brown']:
    time_polys = lrt_time_polys if base_group in ['lrt', 'brown'] else metro_time_polys
    def filter_poly(polygon):
        line = polygon['properties']['LINE'].split('-')[0]
        return line in combos[base_group]
    for time, polys in time_polys.items():
        filtered = filter(filter_poly, polys)
        unioncache[time][base_group] = shapely.ops.unary_union([poly['poly'] for poly in filtered])

print('unionized')

for combo, combo_filter in combos.items():
    print(f'now running for {combo}')
    geojson_features = []
    
    for time, fc in zip(reversed(trip_times), iso_colors):
        print(f'generating {time} minutes geojson features')
        filtered = []
        for group in combo.split('_'):
            i = 0
            for poly in unioncache[time][group]:
                filtered.append(poly)
                i += 1
            print(f'{group} added {i} polygons')
        print(f'unionizing {len(filtered)} polygons now')
        union = shapely.ops.unary_union(filtered)
        for polygon in tqdm(union):
            geojson_features.append({"type": "Feature", "properties": { "time": time, "color": fc }, "geometry": get_geojson_geometry(polygon)})

    geojson = { "type": "FeatureCollection", "features": geojson_features }
    with open(f'./tlv_{combo}_isochrones_merged.geojson', 'w') as f:
        json.dump(geojson, f)


unionized
now running for lrt
generating 15 minutes geojson features
lrt added 3 polygons
unionizing 3 polygons now


  0%|          | 0/3 [00:00<?, ?it/s]

generating 10 minutes geojson features
lrt added 11 polygons
unionizing 11 polygons now


  0%|          | 0/11 [00:00<?, ?it/s]

generating 5 minutes geojson features
lrt added 51 polygons
unionizing 51 polygons now


...

  0%|          | 0/3 [00:00<?, ?it/s]

generating 10 minutes geojson features
brown added 6 polygons
unionizing 6 polygons now


  0%|          | 0/6 [00:00<?, ?it/s]

generating 5 minutes geojson features
brown added 21 polygons
unionizing 21 polygons now


  0%|          | 0/21 [00:00<?, ?it/s]